# Helpers

In [29]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.interpolate import griddata
 

def visualize_stretch(points, stretch_function, output_path):
    xcoords = points[:, 0]
    ycoords = points[:, 1]
    zcoords = points[:, 2]
    X, Y = np.meshgrid(xcoords * 2, ycoords * 2)
    Z = zcoords
    
    xcoords_stretched = np.array([])
    ycoords_stretched = np.array([])
    
    for x, y in zip(xcoords, ycoords):
        xstretched, ystretched = stretch_function(x, y)
        xcoords_stretched = np.append(xcoords_stretched, xstretched)
        ycoords_stretched = np.append(ycoords_stretched, ystretched)

    XStretched, YStretched = np.meshgrid(xcoords_stretched, ycoords_stretched)
    
    # Create a scatter plot for valid points
    fig = go.Figure()

    # Original surface
    fig.add_trace(go.Surface(z=Z, x=X, y=Y, colorscale='Viridis', showscale=True, name='Original Surface'))
    
    # Stretched surface
    fig.add_trace(go.Surface(z=Z, x=XStretched, y=YStretched, colorscale='Cividis', showscale=True, name='Stretched Surface'))

    # Scatter plot for original points
    fig.add_trace(go.Scatter3d(x=xcoords, y=ycoords, z=zcoords, mode='markers', marker=dict(size=5, color='blue'), name='Original Points'))

    # Scatter plot for stretched points
    fig.add_trace(go.Scatter3d(x=xcoords_stretched, y=ycoords_stretched, z=zcoords, mode='markers', marker=dict(size=5, color='red'), name='Stretched Points'))

    fig.update_layout(scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='cube' 
    ))

    fig.show()
    fig.write_image(output_path)
    print(f"Figure saved as {output_path}")

### 2D displacement

#### strecthing

In [10]:
def simple_strecth(x, y):
    x = 2 * x
    y = 1.5 * y
    return x, y

In [21]:
# Define the z-value
z_value = 5.0

# Define the vertices of the polygon (square in this case)
points = np.array([
    [0, 0, z_value],  # Bottom-left
    [1, 0, z_value],  # Bottom-right
    [1, 1, z_value],  # Top-right
    [0, 1, z_value],  # Top-left
    [0, 0, z_value]   # Closing the polygon
])

In [30]:
visualize_stretch(points = points, stretch_function=simple_strecth, output_path="simple_stretch.png")

Figure saved as simple_stretch.png


#### Shear

In [31]:
def shear(x, y):
    x = x + y 
    y = 0.5 * x + y
    return x, y

In [32]:
visualize_stretch(points = points, stretch_function=shear, output_path="shear.png")

Figure saved as shear.png


#### General Deform

In [42]:
def deform1(x, y):
    x = 1.3 * x - 0.375 * y
    y = 0.75 * x + 0.65 * y 
    return x, y

In [34]:
visualize_stretch(points = points, stretch_function=deform1, output_path="shear.png")

Figure saved as shear.png


### Deformation Gradient

In [47]:
import autograd.numpy as np
from autograd import jacobian

def deform(coords):
    x, y = coords
    x_out = 1.3 * x - 0.375 * y
    y_out = 0.75 * x + 0.65 * y
    return np.array([x_out, y_out])

def calculate_deformation_tensor(deform_function):
    gradient_function = jacobian(deform_function)
    identity_matrix = np.eye(2)
    return gradient_function + identity_matrix

F = calculate_deformation_tensor(deform) 
coords = np.array([1.0, 1.0])
F(coords)

array([[ 1.3  , -0.375],
       [ 0.75 ,  0.65 ]])

In [41]:
calculate_deformation_tensor(simple_strecth)

AttributeError: 'function' object has no attribute 'jacobian'